## Load necessary modules

In [1]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# use this to change which GPU to use
gpu = 0

# set the modified tf session as backend in keras
setup_gpu(gpu)

Using TensorFlow backend.


1 Physical GPUs, 1 Logical GPUs


## Load RetinaNet model

In [2]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = os.path.join('413/424detect.h5')

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
#model = models.convert_model(model)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'c', 1: 'gzp', 2: 'jdk', 3: 'jzp', 4: 'ljt',5:'nnh',6:'wsz',7:'xjp',8:'ym'}

tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 , -14.25438 ,  28.50876 ,  14.25438 ],
       [-35.918785, -17.959393,  35.918785,  17.959393],
       [-16.      , -16.      ,  16.      ,  16.      ],
       [-20.158737, -20.158737,  20.158737,  20.158737],
       [-25.398417, -25.398417,  25.398417,  25.398417],
       [-11.313708, -22.627417,  11.313708,  22.627417],
       [-14.25438 , -28.50876 ,  14.25438 ,  28.50876 ],
       [-17.959393, -35.918785,  17.959393,  35.918785]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-45.254833, -22.627417,  45.254833,  22.627417],
       [-57.01752 , -28.50876 ,  57.01752 ,  28.50876 ],
       [-71.83757 , -35.918785,  71.83757 ,  35.918785],
       [-32.      , -32.      ,  32.      ,  32.      ],
       [-40.317474, -40.317474,  40.317474,  40.317474],
       [-50.796833, -50.796833,  50.7

C:\Users\steve\Anaconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


## Run detection on example

In [4]:
txtPath = '413/txts/'
imgPath = '413/img/'
# load image
for img in os.listdir(imgPath):
    image = read_image_bgr(imgPath + img)
    print(os.path.splitext(img)[0])
    f = open(txtPath +  os.path.splitext(img)[0]  + '.txt', 'w')
    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)

    # correct for image scale
    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.15:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
        print(score)
        print(labels_to_names[label])
        print(str(int(box[0])))

        f.write(labels_to_names[label] + ' ' + str(score) + ' ' + str(int(box[0])) +  ' ' + str(int(box[1]))  + ' ' + str(int(box[2])) +  ' ' + str(int(box[3])))
        f.write('\n')


1578190631285i0o11wqixlxc
processing time:  0.14998316764831543
0.46665525
jzp
256
0.43444395
jzp
60
0.4340181
jzp
434
0.40076724
xjp
307
0.3616968
jzp
278
0.36076137
xjp
39
0.33389115
ljt
20
0.30496025
jzp
388
0.27056938
jzp
343
0.26825604
xjp
234
0.24032867
jzp
250
0.23881018
xjp
413
0.22348306
jzp
254
0.20669797
jzp
351
0.20661078
jdk
59
0.20650679
jzp
262
0.18920642
jzp
206
0.18804196
xjp
239
0.17900911
xjp
21
0.16204444
xjp
271
0.16111389
jdk
437
0.15786567
gzp
604
1578190631285i0o11wqixlxc.jpg_0_1032
processing time:  0.11368083953857422
0.48421955
jzp
59
0.44684184
jzp
253
0.44126964
xjp
310
0.39488918
xjp
39
0.37321803
jzp
434
0.32533303
jzp
281
0.30923697
ljt
17
0.30362362
jzp
345
0.28844112
xjp
409
0.27356303
xjp
233
0.25357231
jzp
387
0.25120145
jzp
247
0.21071453
jzp
259
0.20720994
jzp
207
0.19592845
xjp
20
0.18908371
jdk
59
0.18883969
xjp
237
0.18599443
jzp
342
0.1734754
xjp
19
0.16109814
jzp
258
0.15977536
xjp
271
0.15831561
gzp
437
0.15425864
xjp
84
1578352177399x361kszm

1578816553161lt318mm4nc8
processing time:  0.10750389099121094
0.538429
jzp
297
0.45859036
jzp
549
0.36187673
ljt
63
0.3038946
jdk
480
0.27453187
jzp
334
0.25773123
jzp
284
0.2557722
ym
554
0.20339376
jzp
499
0.18343683
wsz
554
1578874332475kvi740bma
processing time:  0.09594464302062988
0.9061243
jdk
435
0.43298054
jdk
239
0.42631146
jdk
285
0.39251602
jzp
222
0.34261537
jzp
212
0.2881685
jdk
266
0.2338519
jzp
278
0.20353968
jzp
248
0.18963908
jzp
391
0.18412934
jzp
282
0.17273891
nnh
439
0.16332486
wsz
489
0.15081376
jdk
224
1578874699390b4ttxzlks9ym
processing time:  0.10844182968139648
0.86226064
jdk
627
0.7567333
jdk
23
0.70179594
jzp
94
0.6403688
jzp
172
0.4088192
jzp
57
0.3168598
gzp
402
0.30424044
jzp
470
0.29635254
jzp
203
0.2957667
jzp
532
0.28340468
jzp
29
0.2827198
jzp
13
0.2744918
jzp
177
0.27225733
jzp
57
0.27079615
jzp
379
0.2669869
xjp
82
0.2656012
jzp
260
0.25219893
jzp
660
0.24161105
jzp
57
0.23041245
jzp
632
0.22917725
jzp
101
0.22829145
jzp
344
0.22654973
jzp
255
0.

1579310481083w7hy4ncezx
processing time:  0.10756278038024902
0.61180675
jzp
46
0.5213248
jzp
141
0.44401085
jzp
380
0.41158485
ym
456
0.33829853
jzp
16
0.3146321
jzp
356
0.26468682
jzp
92
0.25448206
jzp
58
0.24888319
jzp
588
0.24116948
jzp
131
0.21962497
jzp
243
0.20362757
jzp
61
0.2028854
xjp
108
0.19748755
ym
412
0.19337851
jzp
124
0.19059749
jdk
72
0.18879367
xjp
94
0.1791543
xjp
379
0.17483304
xjp
67
0.17414013
xjp
369
0.172972
jzp
418
0.16897582
jzp
423
15793165523109yohdazxxrq
processing time:  0.1205744743347168
0.6095572
jzp
456
0.428723
xjp
154
0.38682315
jdk
237
0.34503856
xjp
386
0.25510505
gzp
406
0.25008696
ljt
42
0.2451542
jdk
521
0.22885396
xjp
134
0.2146355
jzp
410
0.20373511
jzp
448
0.19265094
xjp
56
0.19178262
c
314
0.19158112
jzp
235
0.18848781
gzp
371
0.1848826
ym
153
0.17740022
ljt
122
0.17249028
gzp
342
0.17035712
jzp
440
0.15976049
jzp
314
0.15879306
jzp
228
0.15396635
xjp
338
0.15030894
c
108
1579319583138307w7lsxsozv
processing time:  0.09957742691040039
0.797

0.2234311
jzp
37
0.20636371
jzp
42
0.20631257
jzp
35
0.19634835
xjp
421
0.19604065
jzp
208
0.19065459
wsz
311
0.18293591
jzp
214
0.17914848
xjp
659
0.17833775
jzp
107
0.17478926
jzp
23
0.17455356
ym
315
0.17271586
ym
255
0.16959308
jzp
25
0.1590642
jzp
635
0.15590364
jzp
391
0.15417661
jzp
238
0.15102352
jzp
607
0.15066107
jzp
327
15795673339713v20qxkd3o
processing time:  0.12505173683166504
0.327055
jzp
265
0.2964384
c
124
0.28842175
c
116
0.25651684
nnh
116
0.23099288
xjp
125
0.1837661
gzp
222
0.18261869
c
368
0.16347383
nnh
121
0.15977547
nnh
275
0.15787531
ym
245
0.15107813
ljt
130
0.15080996
wsz
142
15795673339713v20qxkd3o.jpg_0_2223
processing time:  0.12106084823608398
0.35530412
c
139
0.34757182
nnh
139
0.33015263
jzp
265
0.24961615
xjp
142
0.24089502
c
150
0.23781489
ym
246
0.21581346
gzp
223
0.21448974
c
373
0.2089668
nnh
288
0.2051326
wsz
167
0.1989905
nnh
334
0.19412224
c
422
0.185189
nnh
130
0.18094227
c
258
0.17093083
c
552
0.16183084
xjp
364
0.15546687
wsz
277
0.15386534

1579748655186essd16itraf
processing time:  0.09518289566040039
0.42925993
jzp
372
0.29928762
xjp
361
0.28768617
jzp
608
0.24287744
jdk
443
0.21174426
jzp
449
0.2011753
xjp
248
0.19898975
ym
13
0.19668354
xjp
0
0.16788562
jzp
351
0.163761
c
308
0.15149866
jzp
248
1579754438343akfiba9xff04
processing time:  0.10412931442260742
0.5811565
jzp
362
0.44648725
jzp
337
0.420707
jzp
217
0.32515946
jzp
247
0.26317376
ljt
97
0.25550267
jzp
225
0.23046036
jzp
238
0.21472558
jzp
224
0.19421308
jzp
211
0.18193628
jzp
317
0.18176973
jzp
263
0.18062152
jdk
339
0.16426826
jzp
300
0.15070198
jdk
355
15797562192193en7i9m9u
processing time:  0.09178900718688965
0.71242213
jdk
218
0.57246995
jdk
364
0.38674447
jdk
376
0.3591919
jdk
432
0.31652337
ljt
54
0.30921587
gzp
206
0.29910445
jdk
173
0.27754852
jdk
216
0.26644516
jdk
229
0.20479274
jzp
229
0.16667157
jdk
178
1579760172604i12b56f8o60
processing time:  0.11093974113464355
0.82533544
jdk
244
0.24506287
ljt
56
0.1965217
nnh
81
0.19337234
c
151
0.1734899

...

processing time:  0.10858821868896484
0.57897377
wsz
268
0.32466713
wsz
431
0.3096798
ljt
59
0.29503685
jdk
270
0.2396714
wsz
283
0.22086449
nnh
263
0.19764757
jdk
342
0.18316351
xjp
163
0.18294662
xjp
241
0.178493
nnh
406
0.17071809
wsz
245
1585012366709qtiho4k0olc.jpg_0_2173
processing time:  0.11892271041870117
0.3893267
jdk
131
0.3382428
c
90
0.31296632
jzp
129
0.2803932
c
153
0.25596312
xjp
77
0.22211881
c
70
0.21144512
xjp
314
0.17042641
c
190
0.16380014
jdk
330
0.16023394
xjp
43
0.15864949
xjp
331
0.15762442
c
333
1585012443758obihtte61yy5
processing time:  0.09795951843261719
0.18220042
wsz
82
0.15066372
wsz
468
1585013239166b4bzzr3tnv0c
processing time:  0.1170501708984375
0.2823522
ljt
69
0.17016314
wsz
186
0.15254594
ljt
1
15850133776038in44hsa6sqe.jpg_0_9386
processing time:  0.10293436050415039
0.71223414
xjp
22
0.3950236
jzp
4
0.31287885
jzp
419
0.2352018
ym
4
0.2029007
gzp
24
0.19937216
xjp
2
0.19137542
jdk
13
0.18963994
xjp
58
0.18898107
jdk
419
0.16926812
xjp
65
0.1675

processing time:  0.11278486251831055
0.4193552
c
140
0.3527096
c
180
0.24310488
c
302
0.23751451
c
261
0.22217116
c
126
0.2184289
xjp
46
0.21061414
ljt
36
0.20205037
xjp
134
0.19089162
c
110
0.18954591
c
576
0.18742546
c
359
0.1833712
gzp
613
0.16545565
c
389
1585108139155oaer9uuy57z7
processing time:  0.10728645324707031
0.49033856
wsz
39
0.36683404
jdk
36
0.35998178
nnh
31
0.23569131
wsz
165
0.20383357
nnh
48
0.17470346
xjp
44
0.17282428
jdk
406
0.17257851
ljt
37
0.1659991
c
46
0.16087212
wsz
408
1585108894782dy472owtid
processing time:  0.10590291023254395
0.50885355
nnh
402
0.39118868
wsz
418
0.3863227
wsz
65
0.34988314
nnh
453
0.32928503
wsz
466
0.32918924
jdk
440
0.31724817
jdk
464
0.25428542
wsz
323
0.24150755
wsz
328
0.24064691
ym
70
0.17885096
xjp
289
0.16992582
xjp
66
0.16182595
nnh
72
0.15914351
nnh
289
0.15749192
c
261
0.15033372
jdk
66
15851119541099cqli1uhum3t.jpg_0_1498
processing time:  0.1042935848236084
0.5636578
wsz
424
0.47740558
nnh
424
0.346531
nnh
497
0.23877569

0.51477456
ljt
59
0.29895684
wsz
287
0.29884025
wsz
263
0.22670953
wsz
556
0.22289696
wsz
414
0.22264154
wsz
333
0.18589184
ljt
37
0.17036341
jdk
287
0.15917838
wsz
389
0.15103664
wsz
532
1585273560974mlce499d36b6
processing time:  0.12220072746276855
0.5943172
wsz
455
0.39294153
wsz
629
0.39185628
wsz
454
0.3550944
wsz
494
0.27945915
jdk
459
0.23740481
wsz
509
0.22821899
nnh
458
0.22782896
wsz
573
0.22583781
jdk
514
0.21447283
ljt
106
0.19414297
ym
466
0.18860793
xjp
454
0.16780536
nnh
199
0.16735429
jdk
696
0.16040075
wsz
283
0.15933415
wsz
251
0.15320738
ljt
49
1585282505572kdzto092q48.jpg_0_2029
processing time:  0.09712934494018555
0.81811035
jdk
55
0.5852563
jdk
533
0.4169324
xjp
222
0.3567557
jzp
182
0.30234784
jdk
448
0.24986406
jdk
163
0.23643444
xjp
59
0.23426262
jdk
407
0.23091927
jdk
225
0.21717337
jdk
165
0.21102028
jdk
131
0.20960702
jdk
118
0.20262894
jdk
60
0.17732026
jdk
283
0.17022999
jdk
214
1585283806710y3rbmwhin2ae
processing time:  0.09742069244384766
0.28494555
w

processing time:  0.11046457290649414
0.33988303
wsz
247
0.27793998
wsz
465
0.2743277
ljt
20
0.2531681
nnh
47
0.24136448
c
327
0.21813662
jzp
568
0.21576369
ljt
110
0.19654386
nnh
473
0.18381368
c
48
0.18262957
jdk
347
0.17163533
ljt
65
0.16568574
nnh
5
0.15172285
nnh
241
1585528900537vinff2b0yqf
processing time:  0.09665346145629883
0.90379477
jdk
329
0.8972881
jdk
6
0.36537498
xjp
59
0.3234172
jdk
95
0.3113761
gzp
610
0.28901237
wsz
98
0.286774
xjp
90
0.22724162
xjp
158
0.22075163
jzp
59
0.21617036
wsz
343
0.20894495
jdk
21
0.18973751
wsz
2
0.17871179
jdk
220
0.17655109
xjp
26
0.15862934
xjp
293
1585549549088yhhe2bhl1f7s
processing time:  0.09805750846862793
0.5286165
ljt
57
0.49848124
xjp
112
0.4661312
jzp
354
0.40878832
jzp
274
0.3879934
ljt
84
0.34932476
xjp
61
0.3487169
jzp
291
0.24852608
xjp
188
0.20528868
ym
354
0.20114304
c
59
0.1869952
jzp
87
0.1840446
jzp
201
0.18238224
jzp
260
0.17862372
xjp
157
0.16249178
c
112
0.1623683
xjp
217
0.16142263
jzp
138
0.1536905
gzp
427
1585549

0.17395917
jzp
294
0.16657427
jzp
328
1586053283585n63scocunw45
processing time:  0.09339785575866699
0.7417408
jdk
538
0.44862783
xjp
150
0.40236518
c
31
0.3503593
jdk
137
0.34326464
c
44
0.3015549
xjp
48
0.29515952
xjp
44
0.25603592
c
39
0.24863139
xjp
234
0.24810164
jzp
121
0.24514355
c
164
0.24074198
c
48
0.23031211
c
19
0.21796054
c
58
0.20911303
c
217
0.19052821
nnh
178
0.18748413
xjp
410
0.18382248
c
88
0.18370423
wsz
118
0.17124726
c
135
0.16317801
jdk
519
0.1585453
xjp
49
0.15771191
jdk
231
0.15745209
ym
118
0.1566273
gzp
238
0.15582024
jzp
238
0.1545911
nnh
246
0.1535629
ym
52
0.15286233
c
130
1586073691596astid3kztte7
processing time:  0.10196256637573242
0.67663527
jdk
528
0.6633082
jdk
606
0.56432664
ljt
103
0.45052975
jdk
564
0.41417104
xjp
247
0.36912087
jzp
249
0.34417087
xjp
226
0.23720539
ym
250
0.22453068
jdk
500
0.22206877
xjp
196
0.21207851
ljt
198
0.18981324
c
183
0.18389925
c
227
0.16615929
c
250
0.16347969
xjp
155
0.15857477
c
235
0.1550906
ljt
129
0.1514007
gzp